# Fine-tuning com LoRA

### Autenticação

In [ ]:
from scripts.authentcation import authenticate_huggingface

authenticate_huggingface()

### Preparação dos dados

In [ ]:
from datasets import load_dataset
from scripts.data import generate_training_messages

dataset = load_dataset('marmal88/skin_cancer', split='train')

training_data_size = int(input('Enter the size of the training data: '))
messages = generate_training_messages(dataset, training_data_size)

### Inicialização do LLaMa 3.2

In [ ]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    'unsloth/Llama-3.2-11B-Vision-Instruct',
    load_in_4bit=False,
    use_gradient_checkpointing='unsloth'
)

### Carregamento do modelo para treinamento com PEFT

In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r = 16,
    lora_alpha = 16,
    lora_dropout = 0.1,
    bias = 'none',
    random_state = 3407,
    loftq_config = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
)

### Configuração de treinamento

In [ ]:
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig
from sklearn.model_selection import train_test_split

train_messages, val_messages = train_test_split(
    messages,
    test_size=0.1,
    random_state=3407
)

FastVisionModel.for_training(model)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),
    train_dataset=train_messages,
    eval_dataset=val_messages,
    args=SFTConfig(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        learning_rate=1e-4,
        warmup_ratio=0.03,
        num_train_epochs=5,
        fp16=not is_bf16_supported(),
        bf16=is_bf16_supported(),
        lr_scheduler_type='cosine_with_restarts',
        optim='adamw_torch',
        weight_decay=0.01,
        logging_steps=5,
        save_steps=50,
        eval_steps=50,
        eval_strategy='steps',
        gradient_checkpointing=True,
        seed=3407,
        output_dir='outputs',
        report_to='tensorboard',
        remove_unused_columns=False,
        dataset_text_field='',
        dataset_kwargs={'skip_prepare_dataset': True},
        dataset_num_proc=4,
        max_seq_length=2048
    ),
)

### Treinamento

In [ ]:
trainer_stats = trainer.train()

### Salvamento

In [ ]:
peft_model = trainer.model

model.save_pretrained(f'weights/LLaMA_DERM_LoRA_{training_data_size}_11B')
tokenizer.save_pretrained(f'weights/LLaMA_DERM_LoRA_{training_data_size}_11B')